In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from autogluon.core.metrics import make_scorer
from sklearn.metrics import fbeta_score
from autogluon.tabular import TabularPredictor
import shap
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from functools import partial
if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs: {num_gpus}")
    for i in range(num_gpus):
        device_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {device_name}")
else: 
    print('CPU')

import json

shap.initjs()

In [ ]:
test_size = 0.2
time_limit = 300000

metric = 'f1'
metric = 'balanced_accuracy'

presets = 'best_quality'
presets = 'medium_quality'
presets = 'extreme'

random_seed_base = 42

In [ ]:
def f2_scorer_func(y_true, y_pred):
    return fbeta_score(y_true, y_pred, beta=2, average='binary', zero_division=0)

f2_scorer = make_scorer(
    score_func=partial(fbeta_score, beta=2),
    name='f2',
    greater_is_better=True,
    needs_proba=False
)

In [ ]:
class AutogluonWrapper:
    def __init__(self, predictor, feature_names):
        self.ag_model = predictor
        self.feature_names = feature_names
    
    def predict_binary_prob(self, X):
        if isinstance(X, pd.Series):
            X = X.values.reshape(1,-1)
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X, columns=self.feature_names)
        return self.ag_model.predict_proba(X, as_multiclass=False)

def plot_shap_beeswarm(train_val_df, test_df, predictor, metric, random_seed):
    X_train_val = train_val_df.iloc[:, :-1]
    y_train_val = train_val_df.iloc[:, -1]
    X_test = test_df.iloc[:, :-1]
    y_test = test_df.iloc[:, -1]
    # X = df.iloc[:, :-1]

    feature_names = X_train_val.columns

    print("positive class:", predictor.positive_class)

    med = X_train_val.median()
    ag_wrapper = AutogluonWrapper(predictor, feature_names)
    explainer = shap.KernelExplainer(ag_wrapper.predict_binary_prob, med)
    shap_values_arr = explainer.shap_values(X_test.iloc[:,:])

    shap.summary_plot(shap_values_arr, X_test.iloc[:,:], show=False)
    fig = plt.gcf()
    fig.savefig(f'../AutoGluon Figures/beeswarm_{metric}_{random_seed}.png', dpi=600, bbox_inches='tight')
    plt.close(fig)
    return(np.mean(np.abs(shap_values_arr), axis=0))

def plot_shap_force(train_val_df, test_df, predictor, metric, random_seed):
    X_train_val = train_val_df.iloc[:, :-1]
    y_train_val = train_val_df.iloc[:, -1]
    X_test = test_df.iloc[:, :-1]
    y_test = test_df.iloc[:, -1]
    # X = df.iloc[:, :-1]

    feature_names = X_train_val.columns

    print("positive class:", predictor.positive_class)

    med = X_train_val.median()
    ag_wrapper = AutogluonWrapper(predictor, feature_names)
    explainer = shap.KernelExplainer(ag_wrapper.predict_binary_prob, med)
    shap_values_arr = explainer.shap_values(X_test.iloc[:,:])
    shap.force_plot(explainer.expected_value, shap_values_arr, X_test.iloc[:,:])

def plot_shap_bar(shap_values, shap_values_default_seed, test_df, metric):
    sorted_indices = np.argsort(np.abs(shap_values))[::-1]
    sorted_shap_values = shap_values[sorted_indices]
    sorted_shap_values_default_seed = shap_values_default_seed[sorted_indices]
    feature_names = test_df.columns
    sorted_features = [feature_names[i] for i in sorted_indices]

    fig, ax = plt.subplots(figsize=(8, 5))

    bar_width = 0.35
    y_positions = np.arange(len(sorted_shap_values))

    for i, (val, val_default) in enumerate(zip(sorted_shap_values, sorted_shap_values_default_seed)):
        abs_val = abs(val)
        abs_val_default = abs(val_default)

        if abs_val >= abs_val_default:
            ax.barh(y_positions[i] - bar_width/2, val, bar_width, color='skyblue', label='SHAP Values (Average across all 10 seeds)' if i == 0 else '')
            ax.barh(y_positions[i] + bar_width/2, val_default, bar_width, color='lightcoral', label='SHAP Values (Seed 42)' if i == 0 else '')
        else:
            ax.barh(y_positions[i] + bar_width/2, val_default, bar_width, color='lightcoral', label='SHAP Values (Seed 42)' if i == 0 else '')
            ax.barh(y_positions[i] - bar_width/2, val, bar_width, color='skyblue', label='SHAP Values (Average across all 10 seeds)' if i == 0 else '')

    ax.set_yticks(y_positions)
    ax.set_yticklabels(sorted_features)
    ax.set_xlabel('SHAP Value Average Magnitude')
    ax.set_title(f'Feature Importance Based on Average SHAP Value Magnitude ({metric})')
    ax.invert_yaxis()
    ax.legend()  

    plt.tight_layout()
    plt.savefig(f'../AutoGluon Figures/bar_{metric}.png', dpi=600, bbox_inches='tight')
    plt.close(fig)

In [ ]:
random_seeds = [random_seed_base + 10 * modifier for modifier in range(10)]
metrics = ['balanced_accuracy', 'f1', 'f2']
eval_metrics = ['accuracy', 'balanced_accuracy', 'f1', f2_scorer, 'roc_auc', 'precision', 'recall']

df = pd.read_csv('../Datasets/tibial_slope_2.csv')
df = df.drop(columns=['Subject #'])
df['Sex'] = df['Sex'].map({'F': 0, 'M': 1})

label = 'Injury'
best_model = 'WeightedEnsemble_L2'

summary_dir = f'../AutoGluon Summary'
os.makedirs(summary_dir, exist_ok=True)
figure_dir = f'../AutoGluon Figures'
os.makedirs(figure_dir, exist_ok=True)

for metric in metrics:
    leaderboards_per_metric = []
    shap_values_per_metric = []
    for random_seed in random_seeds:
        print('============================================')
        print(f'Metric: {metric}, Random Seed: {random_seed}')
        print('============================================')
        predictor_dir = f'../AutoGluon Models/AutoGluon_{metric}_{random_seed}'
        
        train_val_df, test_df = train_test_split(df, test_size=test_size, random_state=random_seed, stratify=df[label])
        
        if os.path.exists(predictor_dir):
            predictor = TabularPredictor.load(predictor_dir)
        else:
            predictor = TabularPredictor(
                label=label,
                problem_type='binary',
                eval_metric=metric if metric != 'f2' else f2_scorer,
                path=predictor_dir
            )
            predictor.fit(
                train_data=train_val_df,
                time_limit=time_limit,
                presets=presets,
                num_gpus=1
            )

        lb_test = predictor.leaderboard(test_df, extra_metrics=eval_metrics, silent=True).iloc[:, :10]
        columns = lb_test.columns.tolist()
        new_column_order = [columns[0], 'score_val'] + [col for col in columns[1:] if col not in ['score_val', 'score_test']]
        lb_test = lb_test[new_column_order]
        lb_test = lb_test.rename(columns = {'score_val': f'val {metric}'})
        new_columns_names = [col if i < 2 else f'test {col}' for i, col in enumerate(lb_test.columns)]
        lb_test.columns = new_columns_names

        leaderboards_per_metric.append(lb_test)
        shap_value = plot_shap_beeswarm(train_val_df, test_df, predictor, metric, random_seed)
        plot_shap_force(train_val_df, test_df, predictor, metric, random_seed)
        shap_values_per_metric.append(shap_value)

    print('======================================================')
    print(f'Summary ({metric})')
    print('======================================================')

    all_rows = []
    for idx, leader_board in enumerate(leaderboards_per_metric):
        row = leader_board[leader_board['model'] == best_model]
        val_column = [col for col in leader_board.columns if col.startswith('val')][0]
        row = row.rename(columns = {'model': f'random seed'})
        row['random seed'] = random_seeds[idx]
        row = row.drop(columns='test accuracy')
        all_rows.append(row)
    combined_rows = pd.concat(all_rows, ignore_index=True)
    summary_path = os.path.join(summary_dir, f'AutoGluon_{metric}_seeds.csv')
    combined_rows.to_csv(summary_path, index=False)

    all_lb = pd.concat(leaderboards_per_metric, ignore_index=True)
    aggregated_performance = all_lb.groupby('model').agg(
        avg_val_score=(f'val {metric}', 'mean'),
        std_val_score=(f'val {metric}', 'std'),
        avg_testbalanced_accuracy=('test balanced_accuracy', 'mean'),
        std_testbalanced_accuracy=('test balanced_accuracy', 'std'),
        avg_test_F1=('test f1', 'mean'),
        std_test_F1=('test f1', 'std'),
        avg_test_F2=('test f2', 'mean'),
        std_test_F2=('test f2', 'std'),
        avg_test_AUC=('test roc_auc', 'mean'),
        std_test_AUC=('test roc_auc', 'std'),
        avg_test_precision=('test precision', 'mean'),
        std_test_precision=('test precision', 'std'),
        avg_test_recall=('test recall', 'mean'),
        std_test_recall=('test recall', 'std'),        
        count=('model', 'size')
    ).sort_values(by='avg_val_score', ascending=False)
    aggregated_performance = aggregated_performance.reset_index()
    print(f'Summary for {metric}:\n', aggregated_performance)

    summary_path = os.path.join(summary_dir, f'AutoGluon_{metric}.csv')
    aggregated_performance.to_csv(summary_path, index=False)

    shap_values_per_metric = np.array(shap_values_per_metric)
    shap_values_default_seed = shap_values_per_metric[0]
    shap_values_per_metric = np.mean(shap_values_per_metric, axis=0)
    plot_shap_bar(shap_values_per_metric, shap_values_default_seed, test_df, metric)

    predictor_dir = f'../AutoGluon Models/AutoGluon_{metric}_Full'    
    if os.path.exists(predictor_dir):
        predictor = TabularPredictor.load(predictor_dir)
    else:
        predictor = TabularPredictor(
            label=label,
            problem_type='binary',
            eval_metric=metric if metric != 'f2' else f2_scorer,
            path=predictor_dir
        )
        predictor.fit(
            train_data=df,
            time_limit=time_limit,
            presets=presets,
            num_gpus=1
        )
    print(predictor.leaderboard())